In [2]:
import os
import sys
import numpy as np

from scipy.misc import imread
from PIL import Image, ImageOps

def crop_images(base_dir, root_dir, cropped_dir, size):
    cropped_dir_path = os.path.join(base_dir, cropped_dir)
    if not os.path.exists(cropped_dir_path):
        os.makedirs(cropped_dir_path)
        print('Created new directory: ', cropped_dir_path)

    # Walk the directory tree (for the not-yet-cropped images).
    for subdir, dirs, files in os.walk(root_dir):
      for dir in dirs:
        old_dir = os.path.join(root_dir, dir)
        new_dir = os.path.join(cropped_dir, dir)

        # If a corresponding subfolder to hold the cropped images doesn't
        # yet exist, create one.
        if not os.path.exists(new_dir):
          os.makedirs(new_dir)

        for x, y, image_files in os.walk(old_dir):
            for image in image_files:
                image_path = os.path.join(old_dir, image)
                new_image_path = os.path.join(new_dir, image)

                try:
                    im = Image.open(image_path)
                    thumb = ImageOps.fit(im, size, Image.ANTIALIAS)
                    thumb.save(new_image_path, 'JPEG')

                except Exception as e:
                    continue

size = (256, 256)
base_dir = '/Users/mbmay/proj/'
root_dir = base_dir + 'dermis-img'
cropped_dir = base_dir + 'dermis-img-cropped'

#crop_images(base_dir, root_dir, cropped_dir, size)

In [75]:
def prepare_image(image_file):
    return imread(image_file)

def prepare_images():
    images = []
    data = []
    
    for subdir, dirs, files in os.walk(cropped_dir):
        for file in files:
            images.append(os.path.join(subdir, file))
    
    num_images = len(images)
    for i, image in enumerate(images):
        prepared = prepare_image(image)
        data.append(prepared)
        
        print "Added image: %s/%s done" % (i, num_images-1)
        
    return [images, data, num_images]

map_dict = {
    'acne': 0,
    'alopecia': 1,
    'basal_cell_carcinoma': 2,
    'bullous': 3,
    'dermatitis': 4,
    'eczema': 5,
    'exanthem': 6,
    'herpes': 7,
    'keratosis': 8,
    'lupus': 9,
    'melanoma': 10,
    'pigment': 11,
    'psoriasis': 12,
    'rosacea': 13,
    'squamous_cell_carcinoma': 14,
    'tinea': 15,
    'vasculitis': 16
}

# Return the prepared images
image_paths, images, num_images = prepare_images()

# Prepare the label array
labels = [map_dict[f.split('/')[5]] for f in image_paths]

Added image: 0/2868 done
Added image: 1/2868 done
Added image: 2/2868 done
Added image: 3/2868 done
Added image: 4/2868 done
Added image: 5/2868 done
Added image: 6/2868 done
Added image: 7/2868 done
Added image: 8/2868 done
Added image: 9/2868 done
Added image: 10/2868 done
Added image: 11/2868 done
Added image: 12/2868 done
Added image: 13/2868 done
Added image: 14/2868 done
Added image: 15/2868 done
Added image: 16/2868 done
Added image: 17/2868 done
Added image: 18/2868 done
Added image: 19/2868 done
Added image: 20/2868 done
Added image: 21/2868 done
Added image: 22/2868 done
Added image: 23/2868 done
Added image: 24/2868 done
Added image: 25/2868 done
Added image: 26/2868 done
Added image: 27/2868 done
Added image: 28/2868 done
Added image: 29/2868 done
Added image: 30/2868 done
Added image: 31/2868 done
Added image: 32/2868 done
Added image: 33/2868 done
Added image: 34/2868 done
Added image: 35/2868 done
Added image: 36/2868 done
Added image: 37/2868 done
Added image: 38/2868 d

In [76]:
# Flatten images
data = np.array(images).reshape((num_images, -1))

# Convert labels to a numpy array
labels = np.array(labels)

In [80]:
import random
from sklearn import datasets, cross_validation, metrics

import tensorflow as tf
import skflow

random.seed(42)

X_train, X_test, y_train, y_test = cross_validation.train_test_split(data, labels,
    test_size=0.2, random_state=42)

# Build 3 layer DNN with 10, 20, 10 units respecitvely.
classifier = skflow.TensorFlowDNNClassifier(hidden_units=[10, 20, 10],
    n_classes=17, steps=200)

# Fit and predict.
classifier.fit(X_train, y_train)
score = metrics.accuracy_score(y_test, classifier.predict(X_test))
print('Accuracy: {0:f}'.format(score))
print(metrics.classification_report(y_test, classifier.predict(X_test)))

Step #1, avg. loss: 36.14418
Step #21, avg. loss: 258.42651
Step #41, avg. loss: 3.03869
Step #61, avg. loss: 2.91731
Step #81, epoch #1, avg. loss: 2.83185
Step #101, epoch #1, avg. loss: 2.75281
Step #121, epoch #1, avg. loss: 2.70317
Step #141, epoch #1, avg. loss: 2.68648
Step #161, epoch #2, avg. loss: 2.67895
Step #181, epoch #2, avg. loss: 2.66457
Accuracy: 0.139373
             precision    recall  f1-score   support

          0       0.00      0.00      0.00        19
          1       0.00      0.00      0.00         8
          2       0.00      0.00      0.00        49
          3       0.00      0.00      0.00        31
          4       0.14      1.00      0.24        80
          5       0.00      0.00      0.00        70
          6       0.00      0.00      0.00        15
          7       0.00      0.00      0.00        18
          8       0.00      0.00      0.00        51
          9       0.00      0.00      0.00        44
         10       0.00      0.00      0.

/Users/mbmay/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
